# Lab 13: Integer and nonlinear programming

- [Example 1: Portfolio Optimization with Return and Budget Constraints](#example-1-portfolio-optimization-with-return-and-budget-constraints)
- [Example 2: Logistic Regression with L2 Regularization and Norm Constraint](#example-2-logistic-regression-with-l2-regularization-and-norm-constraint)
- [Example 3: Constrained Rosenbrock Function](#example-3-constrained-rosenbrock-function)

In [12]:
from scipy.optimize import minimize
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import numpy as np

## Example 1: Portfolio Optimization with Return and Budget Constraints

We aim to construct a portfolio of three assets that minimizes risk (variance) while achieving a minimum expected return. The optimization problem is:

Minimize  
$$
\frac{1}{2} x^T \Sigma x
$$  
subject to:  
- Budget constraint: $ \sum_{i=1}^3 x_i = 1 $  
- Minimum return constraint: $ \mu^T x \geq R_{\text{min}} $  
- No short-selling: $ 0 \leq x_i \leq 1 $

Where:
- $ \Sigma $ is the covariance matrix of returns
- $ \mu $ is the vector of expected returns
- $ x $ is the vector of investment weights

In [13]:
# Covariance matrix (sigma) and expected returns (mu)
sigma = np.array([[0.1, 0.02, 0.04],
              [0.02, 0.08, 0.01],
              [0.04, 0.01, 0.07]])
mu = np.array([0.12, 0.10, 0.15])
R_min = 0.11  # Minimum acceptable return

# Objective function: 1/2 * x^T sigma x
def objective(x):
    return 0.5 * np.dot(x.T, np.dot(sigma, x))

# Constraints
constraints = [
    {"type": "eq", "fun": lambda x: np.sum(x) - 1},                  # budget
    {"type": "ineq", "fun": lambda x: np.dot(mu, x) - R_min}          # return
]

# Bounds: no short-selling
bounds = [(0, 1)] * 3

# Initial guess
x0 = np.array([1/3, 1/3, 1/3])

# Solve
result = minimize(objective, x0, bounds=bounds, constraints=constraints)

# Output
print("Optimal weights:", result.x)
print("Minimum variance:", result.fun)

Optimal weights: [0.14850215 0.41584976 0.43564809]
Minimum variance: 0.02029702971042159


## Example 2: Logistic Regression with L2 Regularization and Norm Constraint

We perform logistic regression with L2 regularization while constraining the norm of the coefficient vector ( $\beta$ ). The optimization problem is:

**Objective**:  
Minimize  
$$
\sum_{i=1}^n \log(1 + \exp(-y_i \cdot x_i^T \beta)) + \lambda \|\beta\|_2^2
$$  
subject to:  
- $ \|\beta\|_2 \leq \tau $


In [14]:
# Fix: Ensure sum of informative + redundant + repeated < n_features
X, y = make_classification(n_samples=100, n_features=3,
                           n_informative=2, n_redundant=0,
                           random_state=0)

X = StandardScaler().fit_transform(X)
y = 2*y - 1  # Convert labels to {-1, 1}

lmbda = 0.1
tau = 1.5

def objective(beta):
    logits = y * X.dot(beta)
    return np.sum(np.log(1 + np.exp(-logits))) + lmbda * np.sum(beta**2)

def norm_constraint(beta):
    return tau - np.linalg.norm(beta)

x0 = np.zeros(X.shape[1])

result = minimize(objective, x0, constraints={"type": "ineq", "fun": norm_constraint})

print("Optimal beta:", result.x)
print("Objective value:", result.fun)

Optimal beta: [1.48783435 0.07726879 0.17429457]
Objective value: 37.122839550888706


## Example 3: Constrained Rosenbrock Function

The Rosenbrock function is a classic nonlinear optimization test problem. We minimize it with a nonlinear constraint.

**Objective**:  
Minimize  
$$
f(x, y) = (1 - x)^2 + 100(y - x^2)^2
$$  
subject to:  
- $ x^2 + y^2 \leq 1.5 $

In [15]:
# Rosenbrock function
def rosenbrock(v):
    x, y = v
    return (1 - x)**2 + 100*(y - x**2)**2

# Constraint: circular region
def constraint(v):
    x, y = v
    return 1.5 - (x**2 + y**2)

x0 = [0.5, 0.5]

# Solve
result = minimize(rosenbrock, x0, constraints={"type": "ineq", "fun": constraint})

print("Optimal (x, y):", result.x)
print("Function value:", result.fun)

Optimal (x, y): [0.90723396 0.82275545]
Function value: 0.008615650655584985
